# POTUS Election 2020 Analysis (1)

python=3.7

2020 POTUS Election

筛选与2020年美国总统大选有关的帖子和URL处理

In [30]:
import pandas as pd
import seaborn as sns
import re
from joblib import dump, load
from joblib import Parallel, delayed  # 并行计算
from tldextract import extract
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# 输出DataFrame时显示所有的列
pd.set_option('display.max_columns', None)
# 输出DataFrame时每行显示完整的内容
pd.set_option('display.max_colwidth', None)

## 筛选2020年8月17日-2021年1月20日有关的英文帖子

In [16]:
keywords = ["trump", "maga", "MakeAmericaGreatAgain", "Make America Great Again", "sleepy joe", \
            "sleepyjoe", "AmericaFirst", "kag", "pence", "VoteRed2020", \
            "biden", "joe2020", "teamjoe", "kamala", "harris", "demconvention", "demdebate", \
            "BattleForTheSoulOfTheNation", "Battle For The Soul Of The Nation", "VoteBlue2020", \
            "election2020", "elections2020", "debates2020"]

# 处理csv原文件(编号从1到27)
def filter(csv_file_index, keywords, source_path, target_path, result_file_suffix):
    df = pd.read_csv(source_path + str(csv_file_index) + ".csv")  # 从source_path文件夹中取出原文件
    df_f1 = df[df['text'].str.contains('|'.join(keywords), case=False)]  # 对帖文进行关键词检索
    df_f2 = df[df['entities.hashtags'].astype(str).str.contains('|'.join(keywords), case=False)]  # 对话题标签进行关键词检索
    df_cat = pd.concat([df_f1,df_f2], axis=0, join='inner')  # 合并两种检索结果
    df_cat.sort_index(axis=0, ascending=True, inplace=True)  # 恢复帖文顺序
    df_cat.drop_duplicates(subset='id', keep='first', inplace=True)  # 去重
    df_cat.to_csv(target_path + str(csv_file_index) + result_file_suffix + ".csv", index=False)  # 保存

In [18]:
# 并行处理从13到16号csv原文件
begin = 13
end = 16
source_folder = "/mnt/data/Project7/fakenews/csv/"
target_folder = "data/csv_potus_election2020/"
suffix = "_potus_election2020"
Parallel(n_jobs=4)(delayed(filter)(ind, keywords, source_folder, target_folder, suffix) \
                              for ind in range(begin, end+1))

/home/shiqian/miniconda3/envs/py37/lib/python3.7/site-packages/joblib/parallel.py:263: DtypeWarning: Columns (25,27,28,29,31,39,41,42,43,49,65,67,68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  for func, args, kwargs in self.items]
/home/shiqian/miniconda3/envs/py37/lib/python3.7/site-packages/joblib/parallel.py:263: DtypeWarning: Columns (25,27,28,29,31,39,41,42,43,49,65,67,68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  for func, args, kwargs in self.items]
/home/shiqian/miniconda3/envs/py37/lib/python3.7/site-packages/joblib/parallel.py:263: DtypeWarning: Columns (25,27,28,29,31,39,41,42,43,65,67,68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  for func, args, kwargs in self.items]
/home/shiqian/miniconda3/envs/py37/lib/python3.7/site-packages/joblib/parallel.py:263: DtypeWarning: Columns (25,27,28,29,31,39,41,42,43,65,67,68,69) have mixed types.Specify dtype option on import or set 

[None, None, None, None]

In [21]:
# 合并处理结果，并筛选出英文帖子
results = []
for i in range(13, 16+1):
    df = pd.read_csv(target_folder + str(i) + suffix + ".csv")
    results.append(df)
results.reverse()
df_cat = pd.concat(results, axis=0)
df_cat.drop_duplicates(subset='id', keep='first', inplace=True)  # 去重
df_cat = df_cat[df_cat['lang'] == 'en']  # 英文帖子
df_cat.reset_index(drop=True, inplace=True)  # 重置行索引
df_cat.to_csv("data/full_data[topic=POTUS2020].csv", index=False)  # 保存

In [92]:
df_cat.loc[478856:7087323, ['created_at']]  # 478858-7087321

,created_at
478856,2021-01-21T00:00:05.000Z
478857,2021-01-21T00:00:04.000Z
478858,2021-01-20T23:59:59.000Z
478859,2021-01-20T23:59:59.000Z
478860,2021-01-20T23:59:58.000Z
...,...
7087319,2020-08-18T00:00:01.000Z
7087320,2020-08-18T00:00:01.000Z
7087321,2020-08-18T00:00:01.000Z
7087322,2020-08-17T23:59:56.000Z


In [97]:
data = df_cat.loc[478858:7087321, :]
data.to_csv("data/full_data[topic=POTUS2020].csv", index=False)

In [98]:
# 统计用户数
# 帖子作者
authors = set(data[data['author.username'].notna()]['author.username'])
# 被回复者
in_reply_to_users = set(data[data['in_reply_to_username'].notna()]['in_reply_to_username'])
# 被转推者
retweeted_users = set(data[data['retweeted_username'].notna()]['retweeted_username'])
# 被引用者
quoted_users = set(data[data['quoted_username'].notna()]['quoted_username'])
# 涉及到的全部用户
all_users = authors | in_reply_to_users | retweeted_users | quoted_users

# 列出各种用户的数量
print("authors:", len(authors))
print("in_reply_to_users:", len(in_reply_to_users))
print("retweeted_users:", len(retweeted_users))
print("quoted_users:", len(quoted_users))
print("all_users:", len(all_users))

authors: 1463058
in_reply_to_users: 184274
retweeted_users: 74934
quoted_users: 21659
all_users: 1532128


## 处理URL

In [2]:
df = pd.read_csv("data/full_data[topic=POTUS2020].csv")

In [4]:
# 取出帖子中的url
def get_url_from_url_list(url_list: str, url_set: set):
    if type(url_list) != str:
        return url_list
    fd = re.findall('"(.+?)"', url_list)
    url_set.update(fd)

url_set = set()
df['entities.urls'].apply(get_url_from_url_list, args=(url_set, ))
len(url_set)

356253

### 短url（长度小于等于23）

In [7]:
short_urls = set()
for a in url_set:
    if len(a) <= 23:
        short_urls.add(a)

dump(short_urls, "pkl/short_urls[topic=POTUS2020][type=set].pkl")
print("number of short urls: ", len(short_urls))

number of short urls:  24924


去国外服务器上跑转换短URL

In [10]:
# 转换后的短URL
df_reverted_urls = pd.read_csv("data/reverted_short_urls[topic=POTUS2020].csv")
df_reverted_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24924 entries, 0 to 24923
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           24924 non-null  object
 1   reverted_url  24924 non-null  object
dtypes: object(2)
memory usage: 389.6+ KB


### 长url（长度大于23）

In [8]:
long_urls = set({})
for a in url_set:
    if len(a) > 23:
        long_urls.add(a)

dump(long_urls, "pkl/long_urls[topic=POTUS2020][type=set].pkl")
print("number of short urls: ", len(long_urls))

number of short urls:  331329


In [9]:
df_long_urls = pd.DataFrame(long_urls, columns=['url'])
df_long_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331329 entries, 0 to 331328
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     331329 non-null  object
dtypes: object(1)
memory usage: 2.5+ MB


### 长短URL合并&标记

In [11]:
df_long_urls['reverted_url'] = df_long_urls['url']
df_urls = pd.concat([df_reverted_urls, df_long_urls], ignore_index=True)

In [12]:
# 从URL中提取域名的函数
from tldextract import extract
def get_domain(url:str) -> str:
    res = extract(url)
    return res[1] + '.' + res[2]

df_urls['domain'] = df_urls['reverted_url'].apply(get_domain)

In [13]:
# 导入三种媒体的域名列表
# fake news media
fn_df = pd.read_csv("data/FakeNewsDomain_from_iffy.news_23.09.06.csv")
# mainstream media
ms_df = pd.read_csv("data/high_credibility_websites_CoVaxxy_ver2.csv")
# debunking media
db_df = pd.read_csv("data/(Merge_ver2)debunking_fact-checking_sites.csv")

In [14]:
# 给网址打标记
fn_set = set(fn_df['Domain'])
ms_set = set(ms_df['site'])
db_set = set(db_df['domain'].dropna())
db_re = '|'.join(db_set)

df_urls['fake_news'] = ''
df_urls['mainstream'] = ''
df_urls['debunking'] = ''

# 多进程地打标记
def url_labeling(urls:pd.DataFrame, media, label:str):
    '''
    label = 'fake_news', 'mainstream' or 'debunking'
    '''
    if label != 'debunking':
        for i in urls.index:
            urls[label][i] = True if urls['domain'][i] in media else False
    else:
        for i in urls.index:
            urls[label][i] = True if re.search(pattern=db_re, string=urls['reverted_url'][i]) else False
            
    return urls[label]

# 多进程打标记
import multiprocessing as mp
from multiprocessing import Pool
pool = Pool(processes=3)
res_fn = pool.apply_async(url_labeling, (df_urls, fn_set, 'fake_news'))
res_ms = pool.apply_async(url_labeling, (df_urls, ms_set, 'mainstream'))
res_db = pool.apply_async(url_labeling, (df_urls, db_re, 'debunking'))
pool.close()
pool.join()

In [ ]:
# 合并标记结果，保存文件
df_urls_labeled = pd.concat([df_urls.loc[:, ['url','reverted_url','domain']], res_fn.get(), res_ms.get(), res_db.get()], axis=1)
df_urls_labeled.to_csv("data/labeled_urls[topic=POTUS2020].csv", index=False)

测试

In [10]:
import pandas as pd
df = pd.read_csv("data/full_data[topic=POTUS2020].csv")

/home/shiqian/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (25,27,28,29,31,39,41,42,43,49,65,67,68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
df2 = df[df['quoted_username'].notna()]

In [15]:
df3 = df[df['in_reply_to_username'].notna()]

In [23]:
df4 = df[df['retweeted_username'].notna()]

In [31]:
df2.loc[:, ['id','referenced_tweets.quoted.id','quoted_username','author.username','text']].head()

,id,referenced_tweets.quoted.id,quoted_username,author.username,text
3,1352043207031484417,1.352017e+18,campbellclaret,TedDitchburnNNP,"This sort of thing does nothing for commonsense...Biden is talking about Trump and the whole USA Fake News.....it's America, it's now &amp; it's there....\nOtherwise you may as well say he was talking about the dodgy dossier ..or anything and everything.. https://t.co/IQbSVfgLJE"
41,1352042921264967680,1.352006e+18,NormOrnstein,naturewins21,"Media needs to take responsibility for their complicity in spreading misinformation and elevating stupid faux "" scandals"" like tan suitgate . After 5 years of letting Trump literally get away with murder, they face a real reckoning. Media must redeem itself as a source of truth https://t.co/4cxkJRY7XU"
64,1352042726964031489,1.352028e+18,sbg1,stowdynoman,Just go over the campaign season and fact check Biden and Harris. Then fact check the news coverage of Hunter laptop story. The news media lied. It hard to fact check many things because the news media just refused to cover them. https://t.co/ttG2UaBTpu
89,1352042568620638209,1.347756e+18,JORGE_C_NY,cousy,"The snake story told EVERYONE. I wrote about it three years ago. The ""fake news"" ignored it. From a psychological point of view, it was the MOST OBVIOUS THING IN THE WORLD: that the snake would KILL THE BEAUTIFUL LADY (America). Projection: not the immigrants, but DONALD TRUMP. https://t.co/eFIEAPepSx"
144,1352042061650948103,1.351847e+18,nytimes,LadyBookworm117,If people feel Trump was everywhere it is because the biased Liberal media put him there. Our 24/7 news cycle has spent the last 4 years promoting Democrats and bashing Trump. The media didn't care about facts as much as click bait and propaganda. Fake news lives! https://t.co/KqU33zXPv6


In [34]:
df3.loc[:, ['id','referenced_tweets.replied_to.id','in_reply_to_username','author.username','text']].head()

,id,referenced_tweets.replied_to.id,in_reply_to_username,author.username,text
7,1352043184403091457,1.352042e+18,earleyedition,earleyedition,"won't ""lecture"" Trump on inciting insurrection, won't ""lecture"" Craig Kelly on spreading lies and misinformation during a global pandemic. \n\nBut Cricket Australia dropping Australia Day references, now that's a lecturing\nhttps://t.co/okP8aOLkh5"
13,1352043150022500353,1.352027e+18,PatrickByrne,JamesDavidNY,"@PatrickByrne All that shit was bullshit. Vatican,Italy Afghanistan blackout , It was misinformation designed to keep up quelled and run the clock down\nFrom Flynn, Q, parks, to Pompeo, all of them had their part in keeping 80 mill under a fake spell of optimism to run out the clock for Biden"
20,1352043117134938113,1.351935e+18,MSNBC,AmyScharn,@MSNBC I want headlines to read....Fake News falls the people want facts not one sided opinions. Bet you will never force Biden to answer for his sons crimes which he was also played a role in.
49,1352042858472222720,1.352041e+18,AdamGer13430857,AdamGer13430857,"@FrankOw42152567 @graaff_q @PeterlinMary @InesdLC ""vote count in Michigan during the night of Nov. 3 to Nov. 4, Biden received 153,710 votes in Shiawassee County, instead of 15,371,""\n\n ⬆️This from Reuters fact check that says fraud was baseless, Most MSM said same just 24 hours after election without even time to investigate."
53,1352042813836435458,1.351908e+18,Populiste63,GlenSteen,@Populiste63 @nytimes How much damage did #Trump do as #President?\nAmerican economist says Trump was a threat to the World Order at the Vatican.\nCopy &amp; past in browser.\nfile:///C:/Users/SteenG/Downloads/USA%20is%20a%20threat%20to%20world%20order.mp4\nYou'll call this 'fake news' like he told you to do!


In [33]:
df4.loc[:, ['id','referenced_tweets.retweeted.id','retweeted_username','author.username','text']].head()

,id,referenced_tweets.retweeted.id,retweeted_username,author.username,text
1,1352043224383291393,1.352042e+18,jeremynewberger,1brooklyn13,"In other words, that SLEEPY JOE nonsense you and your one term twice impeached golden boy, who incited an insurrection at our US Capitol, have been spinning on your nightly show for years was all as fake news as fake news can be. Your guy couldn't even walk down a ramp."
2,1352043223393435648,1.351985e+18,KimMangone,bresla_linda,Kevin McCarthy congratulated Biden and Harris as if he didn’t spend the past year stoking fear and election misinformation. Retweet if you won’t forget.
4,1352043202761601029,1.352025e+18,JohnSimpsonNews,IITommoII,"In case you didn’t see it, the Washington Post’s Trump fact-checkers say that in his 4 years as President, Donald Trump made 30,573 false or misleading claims. That’s roughly 20 every single day."
5,1352043193207037961,1.352042e+18,jeremynewberger,AliAdair22,"In other words, that SLEEPY JOE nonsense you and your one term twice impeached golden boy, who incited an insurrection at our US Capitol, have been spinning on your nightly show for years was all as fake news as fake news can be. Your guy couldn't even walk down a ramp."
6,1352043191210545160,1.352025e+18,JohnSimpsonNews,GrayMacklin,"In case you didn’t see it, the Washington Post’s Trump fact-checkers say that in his 4 years as President, Donald Trump made 30,573 false or misleading claims. That’s roughly 20 every single day."


In [21]:
print(str(df3['referenced_tweets.replied_to.id'][7]))

1.3520423179228406e+18
